# Notebook com otimização para selecionar ativos

## Instalação e carregamento das bibliotecas de GA em python

In [ ]:
!pip install deap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.7 MB/s eta 0:00:00


In [ ]:
import random
import numpy
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import math
import pandas as pd

## Variáveis Globais

In [ ]:
LISTA_PARAM = []

PESOS_NAIVE = []

MELHOR_INDIVIDUO_NAIVE = []
MELHOR_RETORNO_NAIVE = []
MELHOR_RETORNO_ATIVO_NAIVE = []
MELHOR_UTILIDADE_NAIVE = []


## Função para setar variáveis

In [ ]:
def Setar_variaveis(_Funcoes_Apoio, _Estrategia_Alocacao, df_portfolio, _restricao_QTD_ATIVOS, _restricao_COEFICIENTE_AVERSAO_RISCO):
  #lista_param = []
  LISTA_PARAM.append(_Funcoes_Apoio)
  LISTA_PARAM.append( _Estrategia_Alocacao)
  LISTA_PARAM.append(df_portfolio)
  LISTA_PARAM.append(_restricao_QTD_ATIVOS)
  LISTA_PARAM.append(_restricao_COEFICIENTE_AVERSAO_RISCO)

  return LISTA_PARAM

## Função Objetivo

In [ ]:
def FuncaoObjetivo(individual):

  Funcoes_Apoio = LISTA_PARAM[0]
  Estrategia_Alocacao = LISTA_PARAM[1]
  df = LISTA_PARAM[2]
  restricao_coefiiente_aversao_risco = LISTA_PARAM[4]

  portfolio_selecionado = []
  ativo_selecionado = ''
  for index,valor in enumerate(individual):
    if(valor == 1):
      ativo_selecionado = df.columns[index]
      portfolio_selecionado.append(ativo_selecionado)

  df_portfolio_selecionado = df[portfolio_selecionado]

  # Obtendo a matrix de covariância
  df_portfolio_selecionado_Matrix_Covariancia = Funcoes_Apoio.Gerar_Matrix_Covariancia(df_portfolio_selecionado)

  # Obtendo os retornos por ativo
  portfolio_selecionado_retorno_por_ativo_portfolio = Funcoes_Apoio.Gerar_retorno_por_ativo_portfolio(df_portfolio_selecionado)

  # Resultado Naïve
  restricao_qtd_ativos = LISTA_PARAM[3]
  retorno_Naive = Estrategia_Alocacao.Naive(Funcoes_Apoio,df_portfolio_selecionado,portfolio_selecionado_retorno_por_ativo_portfolio)
  pesos_naive = [1/restricao_qtd_ativos] * int(restricao_qtd_ativos)
  utilidade_naive = Funcoes_Apoio.UtilidadeInvesidor(retorno_Naive, restricao_coefiiente_aversao_risco ,df_portfolio_selecionado_Matrix_Covariancia,pesos_naive)

  df_pesos_naive = df_portfolio_selecionado
  df_pesos_naive = df_pesos_naive.transpose()
  df_pesos_naive = df_pesos_naive.iloc[:,0]
  df_pesos_naive = pd.DataFrame({'':df_pesos_naive.index, '0':pesos_naive[0]})
  df_pesos_naive = df_pesos_naive.set_index('')

  if len(MELHOR_RETORNO_NAIVE) == 0:
    MELHOR_RETORNO_NAIVE.append(retorno_Naive)
    MELHOR_INDIVIDUO_NAIVE.append(individual)
    PESOS_NAIVE.append(df_pesos_naive)
    MELHOR_RETORNO_ATIVO_NAIVE.append(df_portfolio_selecionado)
    MELHOR_UTILIDADE_NAIVE.append(utilidade_naive)


  if (MELHOR_RETORNO_NAIVE[0] < retorno_Naive ):
    MELHOR_RETORNO_NAIVE[0] = retorno_Naive
    MELHOR_INDIVIDUO_NAIVE[0] = individual
    PESOS_NAIVE[0] = df_pesos_naive
    MELHOR_RETORNO_ATIVO_NAIVE[0] = df_portfolio_selecionado
    MELHOR_UTILIDADE_NAIVE[0] = utilidade_naive


  return retorno_Naive,utilidade_naive

## Função de restrição

In [ ]:
# Função de restrição
def FuncaoDeRestricao(individual):
  ehValido = True

  df = LISTA_PARAM[2]
  RESTRICAO_QTD_ATIVOS = LISTA_PARAM[3]

  portfolio_selecionado = []
  ativo_selecionado = ''
  for index,valor in enumerate(individual):
    if(valor == 1):
      ativo_selecionado = df.columns[index]
      portfolio_selecionado.append(ativo_selecionado)

  if sum(individual) != RESTRICAO_QTD_ATIVOS:
    ehValido = False

  return ehValido

## Função para otimizar seleção de ativos

In [ ]:
def otimizar_selecaoativos():

  df = LISTA_PARAM[2]



  creator.create("FitnessMulti", base.Fitness, weights=(1.0,1.0))
  creator.create("Individual", list, fitness=creator.FitnessMulti)

  toolbox = base.Toolbox()
  # Definir o gerador de numeros aleatórios de numeros binários
  toolbox.register("attr_bool", random.randint, 0, 1)
  # Inicialização do cromossomo (quantos genes o cromossomo deve possuir)
  toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, len(df.columns))
  # Registro do individuo na população
  toolbox.register("population", tools.initRepeat, list, toolbox.individual)
  # Registro do nome da função objetivo
  toolbox.register("evaluate", FuncaoObjetivo)
  # Registro da função de penalidade caso o individuo não obedeça as restrições
  toolbox.decorate("evaluate", tools.DeltaPenalty(FuncaoDeRestricao, 0))
  # Registro de qual o tipo de cruzamento deve ser utilizado (cruzamento de 2 pontos)
  toolbox.register("mate", tools.cxTwoPoint) #, indpb=0.8)
  # toolbox.register("mate", CrossoverFunction, icls=creator.Individual, attr_bool_function=toolbox.attr_bool)
  # Registro de qual tipo de mutação deve ser utilizado (probabilidade de um individuo sofrer mutação)
  toolbox.register("mutate", tools.mutFlipBit, indpb=0.3)
  # toolbox.register("mutate", MutationFunction, indpb=0.1)
  # Registro de qual o tipo do método de seleção que será utilizado
  toolbox.register("select", tools.selTournament, tournsize=10)

  pop = toolbox.population(n=1000)                           # inicialização da pop
  hof = tools.HallOfFame(1)                                # melhor indivíduo
  stats = tools.Statistics(lambda ind: ind.fitness.values)  # estatísticas
  stats.register("avg", numpy.mean)
  stats.register("std", numpy.std)
  stats.register("min", numpy.min)
  stats.register("max", numpy.max)
  hof.clear()

  # Definindo sementes iniciais
  individual = gerar_semente_inicial()
  ind1 = creator.Individual(individual)
  pop.append(ind1)

  pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.85, mutpb=0.4, ngen=5, stats=stats,halloffame=hof, verbose=True)


  return hof[0]

## Função para gerar semente inicial

In [ ]:
def gerar_semente_inicial():
  qtdAtivos = len(LISTA_PARAM[2].columns)
  restricao_qtd_ativos = LISTA_PARAM[3]

  individuo = [0]*qtdAtivos
  n = restricao_qtd_ativos

  for _ in range(n):
      if 0 in individuo:
          index_to_replace = random.choice([i for i, x in enumerate(individuo) if x == 0])
          individuo[index_to_replace] = 1
      else:
          break

  return individuo